In [2]:
import numpy as np
import pandas as pd
import requests
import os
import matplotlib.pyplot as plt

In [5]:
dataset_raw = pd.read_csv("dataset_with_hex_and_dis.csv")

In [22]:
dataset_raw

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,osrm_distance,Pick_ID,Drop_ID
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,2009.7,882a1008b3fffff,882a100d69fffff
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,2513.1,882a100d2bfffff,882a1072cdfffff
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,11060.5,882a100d65fffff,882a107289fffff
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,1779.4,882a1072c7fffff,882a107281fffff
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,1615.1,882a100883fffff,882a100895fffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778,2237.3,882a100d29fffff,882a100d27fffff
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655,7637.6,882a100d25fffff,882a100881fffff
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764,10172.9,882a100893fffff,882a107289fffff
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373,1236.0,882a100d29fffff,882a100d61fffff


In [7]:

# Change data types of columns
dataset_raw['pickup_datetime']= pd.to_datetime(dataset_raw['pickup_datetime']) 
dataset_raw['dropoff_datetime']= pd.to_datetime(dataset_raw['dropoff_datetime']) 
dataset_raw = dataset_raw.astype({'vendor_id': 'object'})

In [10]:
insights = pd.DataFrame(columns=["pickup_datetime"])
insights["pickup_datetime"] = dataset_raw['pickup_datetime']
insights["year"] = insights.pickup_datetime.dt.year
insights["month"] = insights.pickup_datetime.dt.month
insights["day"] = insights.pickup_datetime.dt.day
insights["hour"] = insights.pickup_datetime.dt.hour
insights["minute"] = insights.pickup_datetime.dt.minute

In [12]:
insights["Pick_ID"] = dataset_raw['Pick_ID']
insights["Drop_ID"] = dataset_raw['Drop_ID']
insights["passenger_count"] = dataset_raw['passenger_count']
insights.head()

,pickup_datetime,year,month,day,hour,minute,Pick_ID,Drop_ID,passenger_count
0,2016-03-14 17:24:55,2016,3,14,17,24,882a1008b3fffff,882a100d69fffff,1
1,2016-06-12 00:43:35,2016,6,12,0,43,882a100d2bfffff,882a1072cdfffff,1
2,2016-01-19 11:35:24,2016,1,19,11,35,882a100d65fffff,882a107289fffff,1
3,2016-04-06 19:32:31,2016,4,6,19,32,882a1072c7fffff,882a107281fffff,1
4,2016-03-26 13:30:55,2016,3,26,13,30,882a100883fffff,882a100895fffff,1


In [52]:
gkk = insights.groupby(['year', 'month', 'day', 'hour', 'minute', 'Pick_ID', 'Drop_ID'])
insights['count'] = gkk['passenger_count'].transform(len)
new_df = insights[insights['count'].apply(lambda x: x > 1)]
new_df = new_df.sort_values(by='pickup_datetime')
new_df.head()

,pickup_datetime,year,month,day,hour,minute,Pick_ID,Drop_ID,passenger_count,count
970669,2016-01-01 00:50:07,2016,1,1,0,50,882a100d21fffff,882a100d2dfffff,1,2
1218572,2016-01-01 00:50:59,2016,1,1,0,50,882a100d21fffff,882a100d2dfffff,1,2
535452,2016-01-01 01:13:16,2016,1,1,1,13,882a100d21fffff,882a100d63fffff,1,2
24676,2016-01-01 01:13:50,2016,1,1,1,13,882a100d21fffff,882a100d63fffff,1,2
340976,2016-01-01 05:38:55,2016,1,1,5,38,882a107259fffff,882a100d25fffff,2,2


In [53]:
print(new_df.to_string())

            pickup_datetime  year  month  day  hour  minute          Pick_ID          Drop_ID  passenger_count  count
970669  2016-01-01 00:50:07  2016      1    1     0      50  882a100d21fffff  882a100d2dfffff                1      2
1218572 2016-01-01 00:50:59  2016      1    1     0      50  882a100d21fffff  882a100d2dfffff                1      2
535452  2016-01-01 01:13:16  2016      1    1     1      13  882a100d21fffff  882a100d63fffff                1      2
24676   2016-01-01 01:13:50  2016      1    1     1      13  882a100d21fffff  882a100d63fffff                1      2
340976  2016-01-01 05:38:55  2016      1    1     5      38  882a107259fffff  882a100d25fffff                2      2
809922  2016-01-01 05:38:56  2016      1    1     5      38  882a107259fffff  882a100d25fffff                1      2
697384  2016-01-01 10:37:12  2016      1    1    10      37  882a100d65fffff  882a100895fffff                2      2
778123  2016-01-01 10:37:45  2016      1    1    10     